In [ ]:
from typing import Tuple

In [ ]:
# Set input and output folders
# I also define a variable containing the prefix I am interested in
inputPath  = "/data/students/bigdata-01QYD/Lab2/"
outputPath = "res_out_Lab5/" 
outputPath2 = 'res_out_Lab5_Task3'
prefix = "ho"

In [2]:
# Read input data
wordsFrequenciesRDD = sc.textFile(inputPath)

In [3]:
# Task 1

In [4]:
# Keep only the lines containing words that start with the prefix “ho”
selectedLinesRDD = wordsFrequenciesRDD.filter(lambda line: line.startswith(prefix))

In [5]:
# Print on the standard output the following statistics
# - The number of selected lines
numLines = selectedLinesRDD.count()
print("Num. selected lines: "+ str(numLines) ) 

Num. selected lines: 1913


In [6]:
# Print on the standard output the following statistics
# - The maximum frequency (maxfreq) among the ones of the selected lines (i.e., the maximum value of freq in the lines obtained by applying the filter).

# Select the values of frequency
maxfreqRDD = selectedLinesRDD.map(lambda line: float(line.split("\t")[1]))

# Compute the maximu value
maxfreq = maxfreqRDD.reduce(lambda freq1, freq2: max(freq1, freq2) )

# Print maxfreq on the standard output
print("Maximum frequency: "+ str(maxfreq) ) 

Maximum frequency: 39399.0


In [7]:
# Task 2
# Your application must select those lines that contain words with a frequency (freq) 
# greater than 80% of the maximum frequency (maxfreq) computed before.

In [8]:
# Keep only the lines with a frequency freq greater than 0.8*maxfreq.
selectedLinesMaxFreqRDD = selectedLinesRDD.filter(lambda line: float(line.split("\t")[1])>0.8*maxfreq)

In [9]:
# Count the number of selected lines and print this number on the standard output
numLinesMaxfreq = selectedLinesMaxFreqRDD.count()
print("Num. selected lines with freq > 0.8*maxfreq: "+ str(numLinesMaxfreq) ) 

Num. selected lines with freq > 0.8*maxfreq: 3


In [10]:
# Select only the words (first field)
selectedWordsRDD = selectedLinesMaxFreqRDD.map(lambda line: line.split("\t")[0])

In [12]:
# Save the selected words (without frequency) in an output folder (one word per line)
selectedWordsRDD.saveAsTextFile(outputPath)

In [ ]:
# Task 3
# Your application must compute all the six groups and determine the number of words belonging to each group
# each group is defined based on word frequency
# group 0: [0, 99]
# group 1: [100, 199]
# group 2: [200, 299]
# group 3: [300, 399]
# group 4: [400, 499]
# group 5: [500, +inf)
def compute_group(line: str) -> Tuple[str, int]:
    fields = line.split('\t')
    freq = int(fields[1])
    if freq >= 500:
        group = 5
    else:
        group = freq // 100
    return (f'Group {group}', 1)

# compute the pair RDD with key = group, value = +1
groupPairRDD = wordsFrequenciesRDD.map(compute_group)

In [ ]:
# use a reduceByKey to sum all the +1 in the value part
countPerGroupPairRDD = groupPairRDD.reduceByKey(lambda v1, v2: v1 + v2)

In [ ]:
# Save groups
countPerGroupPairRDD.saveAsTextFile(outputPath2)